In [ ]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to edit sqlite file
import sqlite3
# Function to clear output
from IPython.display import clear_output

# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config_tutorial.yml') 
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root) 

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an isntance of the CsvCompressor
compressor = compressors.CsvCompressor(config, logger=logger)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)


In [ ]:
def create_flights_table(conn):
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS flights (
        flight_id TEXT PRIMARY KEY,
        icao24 TEXT,
        callsign TEXT,
        departure_airport TEXT,
        arrival_airport TEXT
    );
    '''
    conn.execute(create_table_sql)

def create_state_vectors_table(conn):
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS state_vectors (
        vector_id INTEGER PRIMARY KEY AUTOINCREMENT,
        flight_id TEXT,
        time INT,
        time_normalized INT,
        lat REAL,
        lon REAL,
        geoaltitude REAL,
        baroaltitude REAL,
        heading REAL,
        velocity REAL,
        FOREIGN KEY (flight_id) REFERENCES flights(flight_id)
    );
    '''
    conn.execute(create_table_sql)

def create_station_data_table(conn):
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS weather_stations (
        station_id TEXT PRIMARY KEY,
        lon REAL,
        lat REAL,
        elevation REAL,
        sname TEXT,
        time_domain TEXT,
        archive_begin TEXT,
        archive_end TEXT,
        state TEXT,
        country TEXT,
        climate_site TEXT,
        wfo TEXT,
        tzname TEXT,
        ncdc81 TEXT,
        ncei91 TEXT,
        ugc_county TEXT,
        ugc_zone TEXT,
        county TEXT,
        network TEXT,
        online BOOL,
        sigma REAL
    );
    '''
    conn.execute(create_table_sql)
        
def create_weather_data_table(conn):
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS weather_data (
        weather_id INTEGER PRIMARY KEY AUTOINCREMENT,
        station_id TEXT,
        time INT,
        tmpf REAL,
        relh REAL,
        drct INT,
        sknt REAL,
        sknt_E REAL,
        sknt_N REAL,
        p01i REAL,
        skyc1 TEXT,
        skyc2 TEXT,
        skyc3 TEXT,
        skyc4 TEXT,
        skyl1 INT,
        skyl2 INT,
        skyl3 INT,
        skyl4 INT,
        wxcodes TEXT,
        wx REAL,
        ice_accretion_1hr REAL,
        METAR TEXT,
        FOREIGN KEY (station_id) REFERENCES weather_sations(station_id)
    );
    '''
    conn.execute(create_table_sql)

In [ ]:
# print("Table renamed successfully.")

In [ ]:
route = 'KSFO_KLAX'
data_file = 'KSFO_KLAX_2023-07-01_2023-07-31.csv'
sqlite_file = 'KSFO_KLAX_2023-07-01_2023-07-31.sqlite'

flights = pd.read_csv(f'../data/flight/{route}/{data_file}')
if 'estdepartureairport' in flights.columns:
    flights['departure_airport'] = flights['estdepartureairport']
    flights['arrival_airport'] = flights['estarrivalairport']
    flights.drop(['estdepartureairport', 'estarrivalairport'], axis='columns')

flights['flight_id'] = [f'{row["icao24"]}_{row["firstseen"]}_{row["lastseen"]}_{row["departure_airport"]}_{row["arrival_airport"]}' for _, row in flights.iterrows()]
    

flight_ids = [f[:-4] for f in os.listdir(f'../data/flight/{route}/state_vectors/') if len(f) == 42]
valid_mask = flights['flight_id'].apply(lambda x: x in flight_ids)
flights = flights[valid_mask]

# Connect to SQLite database
conn = sqlite3.connect(f'../data/flight/{sqlite_file}')
create_flights_table(conn)
flights.to_sql('flights', conn, if_exists='replace', index=True)

# Create state_vectors table
create_state_vectors_table(conn)

for i, flight_id in enumerate(flight_ids):
    clear_output(wait=True)
    print(f'flight_id: {flight_id} | {i}/{len(flight_ids)}')
    file = f'../data/flight/{route}/state_vectors/{flight_id}.csv'
    state_vectors = compressor.decode_to_dataframe_from_file(file)
    state_vectors['flight_id'] = [flight_id]*len(state_vectors)
    state_vectors['time_normalized'] = state_vectors['time'] - state_vectors.iloc[0]['time']
    state_vectors.to_sql('state_vectors', conn, if_exists='append', index=False)

conn.close()


In [ ]:
sqlite_file = 'KSFO_KLAX_2023-07-01_2023-07-31.sqlite'

conn = sqlite3.connect(f'../data/flight/{sqlite_file}')

query = 'SELECT * FROM state_vectors LIMIT 1;'

flights = pd.read_sql_query(query, conn)

conn.close()
flights

In [ ]:
sqlite_file = '2023-06-30_2022-08-01.sqlite'
weather_dir = '../data/weather'
stations_csv = 'stations_database.csv'

os.system(f'rm {weather_dir}/../{sqlite_file}')
stations = pd.read_csv(f'{weather_dir}/{stations_csv}', index_col=0)
stations['station_id'] = stations['id']
stations = stations.drop('id', axis='columns')
stations

# Connect to SQLite database
conn = sqlite3.connect(f'{weather_dir}/../{sqlite_file}')
create_station_data_table(conn)
stations.to_sql('weather_sations', conn, if_exists='replace', index=True)

# Create state_vectors table
create_weather_data_table(conn)

files = [f for f in os.listdir(weather_dir) if len(f) == 25]
for i, file in enumerate(files):
    clear_output(wait=True)
    print(f'file: {file} | {i}/{len(files)}')
    weather_data = pd.read_csv(f'{weather_dir}/{file}', index_col=0)
    weather_data['time'] = weather_data['timestamp']
    weather_data['station_id'] = weather_data['station']
    weather_data['METAR'] = weather_data['metar']

    drops = ['timestamp', 'station', 'valid', 'lon', 'lat', 'metar',
     'temperature_model', 'wind_model_E',
           'wind_model_N', 'tmpf_sea_level', 'sknt_E_sea_level',
           'sknt_N_sea_level', 'elevation', 'smps', 'smps_E', 'smps_N', 'tmpc', 'sigma',
            'tmpf_model', 'sknt_E_model', 'sknt_N_model']
    for drop in drops:
        if drop in weather_data.columns:
            weather_data.drop(drop, axis = 'columns' ,inplace=True)
    weather_data.to_sql('weather_data', conn, if_exists='append', index=False)
conn.close()


In [ ]:
# import sqlite3
# import pandas as pd

# from src.common import utils

# conn = sqlite3.connect('../data/weather/1673827200_1685923200.sqlite')

# max_min_times_df = pd.read_sql_query("SELECT MAX(time) as max_time, MIN(time) as min_time FROM weather_data", conn)
# max_min_lats_df = pd.read_sql_query("SELECT MAX(lat) as max_lat, MIN(lat) as min_lat FROM weather_stations", conn)
# max_min_lons_df = pd.read_sql_query("SELECT MAX(lon) as max_lon, MIN(lon) as min_lon FROM weather_stations", conn)

# mid_t = np.mean(max_min_times_df.values)
# mid_lat = np.mean(max_min_lats_df.values)
# mid_lon = np.mean(max_min_lons_df.values)

# t_1 = max_min_times_df['min_time'].values[0] - 1
# t_2 = max_min_times_df['min_time'].values[0] + 3600 - 1

# distance_thresh = 10000

In [ ]:
# query = f"""
#     SELECT wd.*, ws.lat, ws.lon 
#     FROM weather_data wd
#     JOIN weather_stations ws ON wd.station_id = ws.station_id
#     WHERE wd.time BETWEEN {t_1} AND {t_2}
# """
# weather_data_df = pd.read_sql_query(query, conn)

# # conn.close()

# weather_data_df